## Model experiments with the 4-month collection period format

We currently have a model that uses 8 months of training data to generate predictions. We would like to experiment to see if a version of the model which trains on 4 months of user data to predict conversion in the following 2 month period could be viable. 

In [2]:
!mlflow ui

[2024-05-03 16:45:30 -0400] [85931] [INFO] Starting gunicorn 21.2.0
[2024-05-03 16:45:30 -0400] [85931] [INFO] Listening at: http://127.0.0.1:5000 (85931)
[2024-05-03 16:45:30 -0400] [85931] [INFO] Using worker: sync
[2024-05-03 16:45:30 -0400] [85932] [INFO] Booting worker with pid: 85932
[2024-05-03 16:45:30 -0400] [85933] [INFO] Booting worker with pid: 85933
[2024-05-03 16:45:30 -0400] [85934] [INFO] Booting worker with pid: 85934
[2024-05-03 16:45:30 -0400] [85935] [INFO] Booting worker with pid: 85935
^C
[2024-05-03 16:50:23 -0400] [85931] [INFO] Handling signal: int
[2024-05-03 16:50:23 -0400] [85934] [INFO] Worker exiting (pid: 85934)
[2024-05-03 16:50:23 -0400] [85932] [INFO] Worker exiting (pid: 85932)
[2024-05-03 16:50:23 -0400] [85933] [INFO] Worker exiting (pid: 85933)
[2024-05-03 16:50:23 -0400] [85935] [INFO] Worker exiting (pid: 85935)


In [ ]:
import mlflow

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")